In [11]:
import cv2
import os
from math import ceil
import time
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import random
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import matplotlib as mpl

mpl.rcParams['animation.ffmpeg_path'] = r'C:\\ffmpeg\\bin\\ffmpeg.exe'
import matplotlib.gridspec as gridspec

In [12]:
class VideoReader:
    def __init__(self, path):
        self.path = path
        self.cap = cv2.VideoCapture(path)
        self.height = int(self.cap.get(4))
        self.width = int(self.cap.get(3))
        self.it = iter(self)
        self.length = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)

    def __iter__(self):
        while self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret == False:
                break
            # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            yield frame

    def __call__(self):
        return next(self.it)

    def __del__(self):
        self.cap.release()

    def refresh(self):
        self.cap = cv2.VideoCapture(self.path)

In [13]:
vid = VideoReader('vid1.mp4')
print(vid.length, vid.fps)

150 25.0


In [15]:
class MetricArray:
    def __init__(self, path, height, width):
        self.block_height = height
        self.block_width = width
        self.vid = VideoReader(path)
        self.n_height = ceil(vid.height / self.block_height)
        self.n_width = ceil(vid.width / self.block_width)
        self.scores = np.zeros((self.n_height, self.n_width, vid.length))

    def get_subframe(self, frame, i, j):
        return frame[
               i * self.block_height: min(self.vid.height, (i + 1) * self.block_height),
               j * self.block_width: min(self.vid.width, (j + 1) * self.block_width)
               ]

    def split_video(self):
        """
        devide video on n * m subvideos and save each of them in ./tmp/ folder
        :return:
        """

        TMP_DIR = './tmp/'
        if not os.path.exists(TMP_DIR):
            os.mkdir(TMP_DIR)
        for f in os.listdir(TMP_DIR):
            os.remove(TMP_DIR + f)

        fourcc = cv2.VideoWriter_fourcc(*'XVID')

        for i in range(self.n_height):
            for j in range(self.n_width):
                out = cv2.VideoWriter(f'{TMP_DIR}{i}_{j}.mp4', fourcc, vid.fps, (self.block_width, self.block_height))
                for frame in vid:
                    subframe = self.get_subframe(frame, i, j)
                    out.write(subframe)
                out.release()
                vid.refresh()


m = MetricArray('vid1.mp4', 540, 960)
m.split_video()